### 텍스트 조작 예제 : 고급편

### 학습 내용
* Completion과 Chat 메서드 연결
* 회사이름 “Apple” vs 과일이름 "Apple"(문맥 스터핑)
* 사용자 정의 구조를 기반으로 암호 화폐 정보 얻기(문맥 스터핑)
* 리눅스 명령을 사용하는 챗봇 어시스턴트 만들기

### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


### Completion과 Chat 메서드 연결

In [4]:
import os
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))


prompt = "The first programming language to be invented was Plankalkül, which was de\
signed by Konrad Zuse in the 1940s, but not publicly known until 1972 (and not imple\
mented until 1998).   The first widely known and successful high-level programming l\
anguage was Fortran,   developed from 1954 to 1957 by a team of IBM researchers led \
by John Backus.   The success of FORTRAN led to the formation of a committee of scie\
ntists to develop a universal computer language;   the result of their effort was AL\
GOL 58.  Separately, John McCarthy of MIT developed Lisp,  the first language with o\
rigins in academia to be successful. With the success of these initial efforts, prog\
ramming languages became an active topic of research in the 1960s and beyond\n\nTwee\
t with hashtags:"

# 01 트웟 생성
english_tweet = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=prompt,
  temperature=0.5,
  max_tokens=20
)

english_tweet_text = english_tweet.choices[0].text.strip()
print("English Tweet:")
print(english_tweet_text)


# 02 생성된 내용 번역 요청
spanish_tweet = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Translate to Spanish"
    },
    {
      "role": "user",
      "content": english_tweet_text
    }
  ],
  temperature=0.5
)

spanish_tweet_text = spanish_tweet.choices[0].message.content.strip()
print("Spanish Tweet:")
print(spanish_tweet_text)



English Tweet:
#ProgrammingHistory #FirstProgrammingLanguage #Plankalkül #Fortran #ALGOL #
Spanish Tweet:
#HistoriaDeLaProgramación #PrimerLenguajeDeProgramación #Plankalkül #Fortran #ALGOL #


### 회사이름 “Apple” vs 과일이름 "Apple" (문맥 스터핑)

#### 단어 'light'의 품사를 결정하기

In [5]:
prompt = "Determine the part of speech of the word 'light'.\n\n"

result = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=20,
    temperature=1
)

print( result.choices[0].text.strip() )


The word 'light' can function as various parts of speech depending on its usage in a sentence.


#### 힌트(문장)을 주고, light 품사 물어보기

In [9]:
prompt_a = "The light is red. Determine the part of speech of the word 'light'.\n\n"
prompt_b = "This desk is very light. Determine the part of speech of the word 'light'.\n\n"
prompt_c = "You light up my life. Determine the part of speech of the word 'light'.\\n\n"

for prompt in [prompt_a, prompt_b, prompt_c]:
  result = client.completions.create(
           model="gpt-3.5-turbo-instruct",
         prompt=prompt,
         max_tokens=20,
         temperature=0
  )

  print(result.choices[0].text.strip() )


Noun
Adjective
Noun


#### 하나의 의미는 회사, 하나의 의미는 과일

In [10]:
prompt1 = "Huawei:\ncompany\n\nGoogle:\ncompany\n\nMicrosoft:\ncompany\n\nApple:\n"
prompt2 = "Huawei:\ncompany\n\nGoogle:\ncompany\n\nMicrosoft:\ncompany\n\nApricot:\nFruit\n\nApple:\n"

result = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt1,
    max_tokens=20,
    temperature=0,
    stop=["\n", " "]
)

print(result.choices[0].text.strip() )


company


In [11]:
prompt1 = "Huawei:\ncompany\n\nGoogle:\ncompany\n\nMicrosoft:\ncompany\n\nApple:\n"
prompt2 = "Huawei:\ncompany\n\nGoogle:\ncompany\n\nMicrosoft:\ncompany\n\nApricot:\nFruit\n\nApple:\n"

result = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt2,
    max_tokens=20,
    temperature=0,
    stop=["\n", " "]
)

print(result.choices[0].text.strip() )


Fruit


### 사용자 정의 구조를 기반으로 암호 화폐 정보 가져오기(문맥 스터핑)

In [13]:
import os
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))


prompt = """Input: Bitcoin
Output:
BTC was created in 2008, you can learn more about it here: https://bitcoin.org/en/ a\
nd get the latest price here: https://www.coingecko.com/en/coins/bitcoin.
It's all-time high is $64,895.00 and it's all-time low is $67.81.

Input: Ethereum
Output:
ETH was created in 2015, you can learn more about it here: https://ethereum.org/en/ \
and get the latest price here: https://www.coingecko.com/en/coins/ethereum
It's all-time high is $4,379.00 and it's all-time low is $0.43.

Input: Dogecoin
Output:
DOGE was created in 2013, you can learn more about it here: https://dogecoin.com/ an\
d get the latest price here: https://www.coingecko.com/en/coins/dogecoin
It's all-time high is $0.73 and it's all-time low is $0.000002.

Input: Cardano
Output:\n"""

result = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=200,
    temperature=0
)

print(result.choices[0].text.strip() )


ADA was created in 2015, you can learn more about it here: https://cardano.org/ and get the latest price here: https://www.coingecko.com/en/coins/cardano
It's all-time high is $2.45 and it's all-time low is $0.017.


#### prompt에 출력 형식 변경해 보기 - HTML추가

In [14]:
import os
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))


prompt = """Input: Bitcoin
Output:
BTC was created in 2008, you can learn more about it <ahref="https://bitcoin.org/en/">here</a> and get the latest price <a href="https://www.coingecko.com/en/coins/bitcoin">here</a>.
It's all-time high is $64,895.00 and it's all-time low is $67.81.

Input: Cardano
Output:\n"""

result = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=200,
    temperature=0
)

print(result.choices[0].text.strip() )


Cardano was created in 2015, you can learn more about it <a href="https://cardano.org/">here</a> and get the latest price <a href="https://www.coingecko.com/en/coins/cardano">here</a>.
It's all-time high is $2.45 and it's all-time low is $0.017354.


### 리눅스 명령어 도움말을 위한 챗봇 어시스턴트 만들기

In [21]:
import os
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))


prompt = """
Input: 현재 디렉터리의 모든 파일 목록 확인
Output: ls -l

Input: 숨겨진 파일을 포함한 현재 디렉터리의 모든 파일 목록 확인
Output: ls -la

Input: 현재 디렉터리의 모든 파일을 삭제
Output: rm *

Input:  text.txt의 파일의 단어 sun의 발생 빈도
Output: grep -o "sun" test.txt | wc -l

Input:{}
Output:
"""

result = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt.format("Count the number of files in the current directory"),
    max_tokens=200,
    temperature=0
)

print(result.choices[0].text.strip() )


ls -l | wc -l


#### click 패키지 설치

In [17]:
!pip install click==8.1.3

In [ ]:
### 옮긴이_ 원서 코드는 중간에 멈추는 코드가 없어, 실습 코드에 q를 하면 멈추는 코드를 추가함.

In [22]:
import os
from openai import OpenAI
import click

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

_prompt = """
Input: 현재 디렉터리의 모든 파일 목록 확인
Output: ls -l

Input: 숨겨진 파일을 포함한 현재 디렉터리의 모든 파일 목록 확인
Output: ls -la

Input: 현재 디렉터리의 모든 파일을 삭제
Output: rm *

Input:  text.txt의 파일의 단어 sun의 발생 빈도
Output: grep -o "sun" test.txt | wc -l

Input: {}
Output:"""

while True:
    request = input(click.style("Input:", fg="green"))
    if request=='q':
      break
    prompt = _prompt.format(request)
    result = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=0.0,
        max_tokens=100,
        stop=["\n"]
    )

    command = result.choices[0].text.strip()
    click.echo(click.style("Output: ", fg="yellow") + command)
    click.echo()


Input:file2.txt 파일 생성
Output: touch file2.txt

Input:숨겨진 파일을 포함한 디렉터리 모든 파일 확인
Output: ls -a

Input:현재 디렉터리의 모든 파일 삭제
Output: rm -r *

Input:test02.txt에서 'happy' 단어의 발생 빈도 확인
Output: grep -o "happy" test02.txt | wc -l

Input:q


#### ChatGPT가 생성해준 리눅스 명령을 실행하는 프로그램 만들어보기

In [26]:
import os
import openai
import click

from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

__prompt = """
Input: 현재 디렉터리의 모든 파일 목록 확인
Output: ls -l

Input: 숨겨진 파일을 포함한 현재 디렉터리의 모든 파일 목록 확인
Output: ls -la

Input: 현재 디렉터리의 모든 파일을 삭제
Output: rm *

Input:  text.txt의 파일의 단어 sun의 발생 빈도
Output: grep -o "sun" test.txt | wc -l

Input: {}
Output:"""

while True:
    request = input(click.style("Input (type 'exit' to quit): ", fg="green"))
    if request == "exit":
        break
    prompt = __prompt.format(request)
    result = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=0.0,
        max_tokens=100,
        stop=["\n"]
    )
    command = result.choices[0].text.strip()
    click.echo(click.style("Output: ", fg="yellow") + command)
    click.echo(click.style("Execute? (y/n): ", fg="yellow"), nl=False)
    choice = input()
    if choice == "y":
        os.system(command)
    elif choice == "n":
        continue
    else:
        click.echo(click.style("Invalid choice. Please enter 'y' or 'n'.", fg="red"))
    click.echo()


Input (type 'exit' to quit): file3.txt 파일 생성
Output: touch file3.txt
Execute? (y/n): y

Input (type 'exit' to quit): 종료하기
Output: exit
Execute? (y/n): n
Input (type 'exit' to quit): exit


#### try..except 추가하기

In [27]:
import os
import openai
import click
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

_prompt = """
Input: 현재 디렉터리의 모든 파일 목록 확인
Output: ls -l

Input: 숨겨진 파일을 포함한 현재 디렉터리의 모든 파일 목록 확인
Output: ls -la

Input: 현재 디렉터리의 모든 파일을 삭제
Output: rm *

Input:  text.txt의 파일의 단어 sun의 발생 빈도
Output: grep -o "sun" test.txt | wc -l

Input: 새로운 파일 생성 - "test.txt"
Output: touch file1.txt
Input: {} Output:"""

while True:
    request = input(click.style("Input (type 'exit' to quit): ", fg="green"))
    if request == "exit":
        break
    prompt = _prompt.format(request)

    try:
        result = client.completions.create(
        model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            temperature=0.0,
            max_tokens=100,
            stop=["\n"]
        )

        command = result.choices[0].text.strip()
        click.echo(click.style("Output: ", fg="yellow") + command)
        click.echo(click.style("Execute? (y/n): ", fg="yellow"), nl=False)
        choice = input()
        if choice == "y":
            os.system(command)
        elif choice == "n":
            continue
        else:
            click.echo(click.style("Invalid choice. Please enter 'y' or 'n'.", fg="red"))
    except Exception as e:
        click.echo(click.style("The command could not be executed. {} ".format(e), fg="red"))
        pass
    click.echo()


Input (type 'exit' to quit): text2.txt 파일 생성
Output: touch text2.txt
Execute? (y/n): y

Input (type 'exit' to quit): exit
